In [182]:
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve,auc, fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [183]:
RANDOM_SEED = 888

In [184]:
titanic = pd.read_csv('C:/Users/150ho/Desktop/data analyst/4. Машинное обучение/lesson1/archive1/titanic.csv')

In [185]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [186]:
titanic.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [187]:
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [188]:
titanic.loc[titanic.Age.isna(),'Age'] = titanic['Age'].mean()

In [189]:
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [190]:
titanic = titanic[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

### Закодируем пол в число

In [191]:
le = preprocessing.LabelEncoder()

In [192]:
titanic['POL'] = le.fit_transform(titanic['Sex'])

In [193]:
titanic

,Survived,Pclass,Sex,Age,Fare,POL
0,0,3,male,22.000000,7.2500,1
1,1,1,female,38.000000,71.2833,0
2,1,3,female,26.000000,7.9250,0
3,1,1,female,35.000000,53.1000,0
4,0,3,male,35.000000,8.0500,1
...,...,...,...,...,...,...
886,0,2,male,27.000000,13.0000,1
887,1,1,female,19.000000,30.0000,0
888,0,3,female,29.699118,23.4500,0
889,1,1,male,26.000000,30.0000,1


In [194]:
X_train, X_test, y_train, y_test= train_test_split(titanic.drop(['Age', 'Sex'], axis=1), 
                                                  titanic['Age'],
                                                  test_size = 0.3,
                                                  random_state = RANDOM_SEED)

In [195]:
print(X_train.shape,X_test.shape)

(623, 4) (268, 4)


# С помощью деревьев по дефолту

In [196]:
tree_regressor = DecisionTreeRegressor()

In [197]:
tree_regressor.fit(X_train,y_train)

DecisionTreeRegressor()

In [198]:
predict_tree = tree_regressor.predict(X_test)

In [199]:
print('MAE_1 on DecisionTreeRegressor', mean_absolute_error(y_test, predict_tree))

MAE_1 on DecisionTreeRegressor 10.948521894205442


# Подбор гиперпараметров (вручную)

In [200]:
test_deep = []
for deep in range (1,11):
  for leaf in range (1,5):
    clf = DecisionTreeRegressor(max_depth=deep, min_samples_leaf=leaf)
    model = clf.fit(X_train, y_train)
    pred = model.predict(X_test)
    test_deep.append([deep,
                      leaf,
                      mean_absolute_error(y_test, pred),
                      mean_absolute_error(y_train, model.predict(X_train))
                     ])

In [201]:
test_deep = pd.DataFrame(test_deep)
test_deep.columns = ['tree_depth',
                     'leaf',
                     'MAE_TEST',
                     'MAE_TRAIN']

In [202]:
test_deep.sort_values(by='MAE_TEST').head(11)

,tree_depth,leaf,MAE_TEST,MAE_TRAIN
11,3,4,8.615233,8.603972
10,3,3,8.615233,8.603972
8,3,1,8.642487,8.550062
9,3,2,8.642487,8.550062
13,4,2,8.755759,8.430978
15,4,4,8.779625,8.485524
14,4,3,8.796918,8.486132
12,4,1,8.879733,8.390689
17,5,2,8.893686,8.176267
19,5,4,8.917486,8.246015


In [203]:
tree_regressor = DecisionTreeRegressor(max_depth=3, min_samples_split=3)
tree_regressor.fit(X_train,y_train)
predict_tree = tree_regressor.predict(X_test)
print('MAE_2 on DecisionTreeRegressor', mean_absolute_error(y_test, predict_tree))

MAE_2 on DecisionTreeRegressor 8.642486937931768


# GridSearch

In [204]:
tree_para = {'max_depth':[1,2,3,4,5,6,7,8,9,10],
             'min_samples_leaf':[1,2,3,4,5]}

In [205]:
clf_gs = GridSearchCV(DecisionTreeRegressor(), tree_para, cv=3)

In [206]:
clf_gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4, 5]})

In [207]:
clf_gs.best_params_

{'max_depth': 1, 'min_samples_leaf': 5}

# New best params

In [208]:
model_new = DecisionTreeRegressor(**clf_gs.best_params_)

In [209]:
model_new.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=1, min_samples_leaf=5)

In [210]:
new_pred = model_new.predict(X_test)

In [211]:
print('MAE_3 on DecisionTreeRegressor', mean_absolute_error(y_test, new_pred ))

MAE_3 on DecisionTreeRegressor 9.113517707317317


# Сравниваем результаты

MAE_1 on DecisionTreeRegressor 10.948521894205442

MAE_2 on DecisionTreeRegressor 8.642486937931768

MAE_3 on DecisionTreeRegressor 9.113517707317317

In [212]:
# В итоге лучий выбор параметр вручную. Либо что то не так сделал)